In [9]:
import pandas as pd
import lasio
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import numpy as np

In [10]:
las = lasio.read(r"F:\Petrophysics Sabah Basin Malaysia\Priorty Wells_15_8_25\NURI-1RDR1\1000023366_version_8.las")
df = las.df()

In [5]:
f1 = df.reset_index()

sd = pd.DataFrame([])
s = np.arange(f1["DEPTH"].min(),f1["DEPTH"].max(),0.1524)
sd["DEPTH"] = s
for i in f1.columns:
    ff = interp1d(f1["DEPTH"],f1[i],fill_value=np.nan,bounds_error=False)
    sd[i] = ff(s)



In [6]:
sd

,DEPTH,CALI_PPQI,DTC_PPQI,DTS_PPQI,GR_PPQI,NPHI_PPQI,RHOB_PPQI,RHOB_PQI,RS_PPQI,RT_PPQI,TVDSS_PPQI
0,3645.4080,NaN,NaN,NaN,21.80260,0.547200,NaN,NaN,2000.0000,8.951200,3620.367900
1,3645.5604,465.8258,NaN,NaN,21.43675,0.547276,NaN,NaN,2000.0000,8.951401,3620.368101
2,3645.7128,465.8258,NaN,NaN,20.29980,0.662900,NaN,NaN,2000.0000,9.255700,3620.673100
3,3645.8652,466.1636,NaN,NaN,19.24010,0.662900,NaN,NaN,2000.0000,9.255700,3620.673100
4,3646.0176,466.1636,NaN,NaN,19.95620,0.519900,NaN,NaN,1945.6453,9.395200,3620.825000
...,...,...,...,...,...,...,...,...,...,...,...
11745,5435.3460,NaN,196.333714,346.343459,NaN,NaN,NaN,NaN,NaN,NaN,5124.232302
11746,5435.4984,NaN,196.235399,346.435070,NaN,NaN,NaN,NaN,NaN,NaN,5124.352899
11747,5435.6508,NaN,196.137085,346.526680,NaN,NaN,NaN,NaN,NaN,NaN,5124.473497
11748,5435.8032,NaN,196.038770,346.618291,NaN,NaN,NaN,NaN,NaN,NaN,5124.594094


In [7]:
sd.index  =sd["DEPTH"]
sd = sd.drop(columns=["DEPTH"])

In [8]:
sd

,CALI_PPQI,DTC_PPQI,DTS_PPQI,GR_PPQI,NPHI_PPQI,RHOB_PPQI,RHOB_PQI,RS_PPQI,RT_PPQI,TVDSS_PPQI
DEPTH,,,,,,,,,,
3645.4080,NaN,NaN,NaN,21.80260,0.547200,NaN,NaN,2000.0000,8.951200,3620.367900
3645.5604,465.8258,NaN,NaN,21.43675,0.547276,NaN,NaN,2000.0000,8.951401,3620.368101
3645.7128,465.8258,NaN,NaN,20.29980,0.662900,NaN,NaN,2000.0000,9.255700,3620.673100
3645.8652,466.1636,NaN,NaN,19.24010,0.662900,NaN,NaN,2000.0000,9.255700,3620.673100
3646.0176,466.1636,NaN,NaN,19.95620,0.519900,NaN,NaN,1945.6453,9.395200,3620.825000
...,...,...,...,...,...,...,...,...,...,...
5435.3460,NaN,196.333714,346.343459,NaN,NaN,NaN,NaN,NaN,NaN,5124.232302
5435.4984,NaN,196.235399,346.435070,NaN,NaN,NaN,NaN,NaN,NaN,5124.352899
5435.6508,NaN,196.137085,346.526680,NaN,NaN,NaN,NaN,NaN,NaN,5124.473497


In [11]:
import numpy as np
import pandas as pd
import lasio

def export_df_to_las(
    df,
    output_path,
    depth_min=None,          # e.g., 4000 to keep rows >= 4000
    depth_mnemonic="DEPTH",  # or "DEPT" if you prefer LAS-style
    depth_unit="M",
    units=None,              # dict like {"GR_PPQI":"API","RHOB_PPQI":"g/cc"}
    null_value=-999.25,
):
    """
    Export a DataFrame (depth as index) to a standalone LAS 2.0 file.
    Works with older lasio (no add_curve).
    """

    # 1) Select depth range
    df_work = df.copy()
    if depth_min is not None:
        df_work = df_work.loc[df_work.index >= float(depth_min)]

    # 2) Ensure ascending depth
    if not df_work.index.is_monotonic_increasing:
        df_work = df_work.sort_index()

    # 3) Move depth index to a column named depth_mnemonic
    df_reset = df_work.reset_index()
    df_reset.rename(columns={df_reset.columns[0]: depth_mnemonic}, inplace=True)

    # 4) Ensure numeric + fill nulls
    df_reset = df_reset.apply(pd.to_numeric, errors="coerce").fillna(null_value)

    # 5) Build LAS: define curves first (no add_curve used)
    las = lasio.LASFile()
    cols = df_reset.columns.tolist()

    for col in cols:
        col_unit = depth_unit if col == depth_mnemonic else (units.get(col, "") if units else "")
        descr = "Depth" if col == depth_mnemonic else f"{col} log"
        # Define curve header
        las.curves.append(lasio.CurveItem(mnemonic=str(col), unit=col_unit, descr=descr))

    # 6) Attach data (now that curves are defined)
    las.set_data(df_reset.values, names=None)  # names come from the curves we just appended

    # 7) Well info
    depth_series = df_reset[depth_mnemonic].astype(float)
    las.well.STRT.value = float(depth_series.iloc[0])
    las.well.STOP.value = float(depth_series.iloc[-1])
    if len(depth_series) > 1:
        step = float(np.median(np.diff(depth_series.values)))
    else:
        step = 0.0
    las.well.STEP.value = step
    las.well.NULL.value = null_value

    # 8) Write LAS
    las.write(output_path, version=2.0, wrap=False)





In [12]:
# -------- Example usage --------
# keep rows from 4000 m onward, then export
export_df_to_las(
    sd,
    "NURI-1RDR1_Nora.las",
    depth_min=4000,          # remove this arg if you don't want trimming
    depth_mnemonic="DEPTH",  # keep as "DEPTH" to match your DataFrame
    depth_unit="M",
    units=None,              # or provide {"GR_PPQI":"API","RHOB_PPQI":"g/cc", ...}
    null_value=-999.25,
)

In [13]:
import lasio
chk = lasio.read("NURI-1RDR1_Nora.las")
print([c.mnemonic for c in chk.curves])   # should list DEPTH + your columns
print(chk.df().head())

['DEPTH', 'CALI_PPQI', 'DTC_PPQI', 'DTS_PPQI', 'GR_PPQI', 'NPHI_PPQI', 'RHOB_PPQI', 'RHOB_PQI', 'RS_PPQI', 'RT_PPQI', 'TVDSS_PPQI']
           CALI_PPQI   DTC_PPQI   DTS_PPQI   GR_PPQI  NPHI_PPQI  RHOB_PPQI  \
DEPTH                                                                        
4000.0428  314.62449  215.71634  404.20889  65.55763     0.1072    2.60119   
4000.1952  314.30980  216.68370  410.54270  66.35730     0.1072    2.58050   
4000.3476  314.87110  219.55860  410.54270  66.65150     0.1137    2.57220   
4000.5000  314.87110  222.17070  417.25650  67.34340     0.1259    2.56730   
4000.6524  315.45788  222.17298  431.99503  68.68312     0.1239    2.56729   

           RHOB_PQI   RS_PPQI   RT_PPQI  TVDSS_PPQI  
DEPTH                                                
4000.0428    2.5805  21.60327  17.18050   3974.9309  
4000.1952    2.5805  20.49450  17.18050   3974.9309  
4000.3476    2.5716  18.99630  16.41610   3975.0830  
4000.5000    2.5722  17.28470  14.30430   3975.3870